In [12]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
#main
%autoreload
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime
import scipy as sc
from scipy.stats import norm
import math
import utilities as ut
import control as ct
import utilities_2 as ut2

In [20]:
df=pd.read_csv('EUROSTOXX50_2023_Dataset.csv', index_col=0)
#df = df.fillna(method='ffill') 
np_num, np_den=ut.read_our_CSV(df)

In [21]:
alpha=0.95
notional=1e7
delta=1
n_asset=4
weights=np.ones((n_asset,1))/n_asset

In [22]:
returns=np.log(np_num/np_den)

In [24]:
VaR, ES=ut.AnalyticalNormalMeasures(alpha, weights, notional, delta, returns)
VaR, ES

(146813.58536847658, 185434.4799413243)

In [26]:
#CHECK
VaR_check, ES_check=ct.plausibilityCheck(returns, weights, alpha, notional, delta)

[[56938.81775921]
 [47109.33808388]
 [41452.04846453]
 [42170.9708321 ]]


TypeError: cannot unpack non-iterable float object

In [24]:
#Exercise 2
#Parameters
sett_date='2023-01-31'
expiry='2023-04-05'
final_date='2021-02-01'
strike=25
value_ptf=25870000
volatility=0.154
dividend=0.031
alpha_2=0.99
days_VaR=10
rate=0

In [25]:
#We select only the stocks of Vonovia between the settlement date and 2y before
df_ptf2 = df[final_date:sett_date]
df_ptf2 = df_ptf2.loc[:, ['VNAn.DE']]
df_den2 = df['2021-01-29':'2023-01-30']
df_den2 = df_den2.loc[:, ['VNAn.DE']]
np_den2 = df_den2.to_numpy()
np_num2 = df_ptf2.to_numpy()


In [52]:
stockPrice=df_ptf2['VNAn.DE'][len(df_ptf2)-1]
numberOfShares=stockPrice*value_ptf
numberOfPuts=numberOfShares
start = datetime.strptime(sett_date, "%Y-%m-%d")
end = datetime.strptime(expiry, "%Y-%m-%d")
diff = end - start
timeToMaturityInYears=diff.days/365
riskMeasureTimeIntervalInYears=days_VaR/365
NumberOfDaysPerYears=260
logReturns=np.log(np_num2/np_den2)

In [53]:
VaR_MC, ES_MC=ut2.FullMonteCarloVaR(logReturns, numberOfShares, numberOfPuts, stockPrice, strike, rate, dividend,
                            volatility, timeToMaturityInYears, riskMeasureTimeIntervalInYears, alpha,NumberOfDaysPerYears)

TypeError: 'float' object cannot be interpreted as an integer